In [11]:
import random

In [128]:
def generate_balanced_brackets(n):
    brackets = ['(', ')', '[', ']', '{', '}']
    map = {'(': ')', '[': ']', '{': '}'}
    sequence = ''
    stack = []
    
    for i in range(n):
        open_or_close = random.choice([0, 1]) and stack
        if n-i < len(stack) or (not open_or_close and len(stack) == n-i):
            open_or_close = 1
        if open_or_close:
            sequence += map[stack.pop()]
        else:
            bracket = random.choice(['(', '[', '{'])
            sequence += bracket
            if bracket in map:
                stack.append(bracket)
    
    return sequence

def is_balanced(s):
    stack = []
    brackets = {')': '(', '}': '{', ']': '['}
    
    for bracket in s:
        if bracket in brackets:
            if not stack or stack.pop() != brackets[bracket]:
                return False
        else:
            stack.append(bracket)
    
    return not stack

def generate_bracket_sequence(n):
    brackets = ['(', ')', '[', ']', '{', '}']
    sequence = ''
    
    for _ in range(n):
        sequence += random.choice(brackets)
    
    if is_balanced(sequence):
        sequence = generate_bracket_sequence(n)
    
    return sequence

In [177]:
import pandas as pd

# Generate 2000 balanced sequences
balanced_sequences = [generate_balanced_brackets(64) for _ in range(2000)]

# Generate 10000 unbalanced sequences
unbalanced_sequences = [generate_bracket_sequence(64) for _ in range(10000)]

# Create the dataframe
df = pd.DataFrame({'Sequence': balanced_sequences + unbalanced_sequences, 'Balanced': [1] * 2000 + [0] * 10000}, columns=['Sequence', 'Balanced'])

# shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# Print the dataframe
print(df.head())

                                            Sequence  Balanced
0  (]}(]){{[[[)}([]{)[))[){(()}{(})}[{)))[]}){]()...         0
1  )()([]](({){][)[[)]{]](}(}{({[(]]}[][((((]]]{]...         0
2  {(){{}()}}[([[{}]]{})][][[]][(()){[({}{{{()[]{...         1
3  )}]{})[)))}([){)[{({][{(({{{[]}})}{](}(]](){{{...         0
4  [()]){{{}[{[[((]}[]))]][}](({[][{{[[[[{[}))]}]...         0


In [178]:
df.to_csv('balanced_brackets.csv', index=False)